<a href="https://colab.research.google.com/github/MoohShadox/Web_Semantique_Workshop/blob/master/Lab%2003%20-%20OwlReady.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install owlready2
from owlready2 import *
!git clone https://github.com/MoohShadox/Web_Semantique_Workshop

In [0]:
!pip install rdflib
import rdflib

# Lab 02 : OWLReady 
Dans ce lab nous allons étudier le fonctionnement de OWLReady et les possibilités que cette librairie offre, nous aborderons notamment les questions suivantes : 
- Manipuler des ontologies déja existantes
- Comment enrichir une ontologie
- Comment créer et ajouter des individus
- Comment éditer des régles d'inférences 
- Comment exécuter le raisonneur et les différents raisonneurs qui existent (Hermit, Pellet etc..) 
- Comment voire ce que le raisonner a inféré et notamment comment vérifier qu'une classe est inconsistante.
- Comment transformer une ontologie en graphe rdflib afin de pouvoir exécuter des requêtes SPARQL dessus.

## 1- Récupérer et Manipuler des ontologies 
Avant toute chose il faut savoir que si les ontologies peuvent être sérialisées en un fichier .owl ce n'est pas la représentation la plus pratique pour stocker des grands volumes de données, ils serrons plûtot enregistrées dans des quadstore qui ont pour avantage d'être indexées.

Cette première ligne servira donc a fixer le back end ce qui signifie trois chose : 
- Si le fichier existe déja la ligne va automatiquement récupérer les ontologies qui sont dessus
- Si le fichier n'existe pas et que vous n'avez pas encore commencé la construction de l'ontologie, le fichier serra crée.
- Si le fichier n'existe pas mais que vous avez déja commencé a construire l'ontologie (si vous avez exécuté le notebook puis que vous êtes remontés pour executer cette cellule par exemple) ça renverra une erreur.

In [0]:
default_world.set_backend(filename = "file_back3.sqlite3", exclusive = False)

Ensuite, il est possible de charger une ontologie a partir d'un fichier OWL sérialisé en RDF/XML, ce fichier peut provenir de l'exercice précédent qui génére une ontologie avec RDFLib ou directement de protégé.
Dans cette application nous utiliserons une ontologie faite sur protégé.

In [0]:
onto = get_ontology("file:///content/Web_Semantique_Workshop/Amazone_Ontology.owl").load()
!cat /content/Web_Semantique_Workshop/Amazone_Ontology.owl

Maintenant que nous avons récupéré notre ontologie voici quelques exemples de méthodes permettant de l'explorer : 

In [0]:
#Afficher les classes : 
for i in onto.classes():
    print(i)

In [0]:
#Afficher les propriétés : 
for i in onto.object_properties():
    print(i)

In [0]:
for i in onto.individuals():
    print(i)

Nous remarquons que l'ontologie ne contient pas d'individus mais nous allons y remédier dans la prochaine partie.

## 2- Créer des individus
Pour accéder aux classes présentes dans l'ontologie nous pouvons utiliser le générateur de classes qu'on transformera en liste pour l'occasion.

In [0]:
class_article = list(onto.classes())[0]
A = class_article()

In [0]:
for i in onto.individuals():
    print(i)

On peut renommer nos individus sachant que par défaut leur display 

In [0]:
mon_iri = "http://www.semanticweb.org/mohamed/ontologies/2020/3/Amazone_Ont#11122002"
I = list(onto.individuals())[0]
I.iri = mon_iri
for i in onto.individuals():
    print(i)

In [0]:
for i in onto.individuals():
    print(i.namespace, " ", i.name)

In [0]:
L = onto.search(type = class_article)
print(L)

Et cette fois ci on remarquera qu'on a bien un individu qui c'est crée, réexecutais plusieurs fois la première cellule vous pourrez constater que l'instruction crée bien un individu.

In [0]:
relation_alsobought = list(onto.object_properties())[0]

In [0]:
relation_alsobought

Cependant notre petite astuce ne permet pas de créer des relations directement, il faudra associer a notre objet relation un "python_name" qui servira a l'appeler comme une propriété de classe.

In [0]:
relation_alsobought.python_name = "also_bought"
A = class_article()
A.also_bought = [A]
print(A.also_bought)

## 3- Enrichir une ontologie 
Cette fois ci, nous allons essayer de reconstruire une nouvelle ontologie de zéro en utilisant owlready. 
Ce qui est intéressant avec cette façon de faire c'est que nous allons créer des classes Python en même temps que les classes représentant les concepts correspondant dans l'ontologie.

In [0]:
#On crée une nouvelle ontologie 
onto = get_ontology("https://urlinvente/monontologie")

with onto:
    
    #On crée une classe Animal et une classe Plante qui héritent toutes deux de Thing
    #On peut même créer des méthodes dans les classes comme en orienté objet
    class Animal(Thing):
        def presente_toi(self):
            print("je suis un animal !")
        
    class Plante(Thing):
        pass
    
    #On précise que les deux classe sont disjointes
    AllDisjoint([Animal, Plante])
    
    #On crée une propriété qui va avoir comme domain Animal et comme range Thing
    class mange(Animal >> Thing):
        pass
    
    class nemangepas(Animal >> Animal):
        pass
    
    """On crée une dataproperty cette fois sans utiliser la notation abrégée ce qui nous permet d'hériter a la fois
    de DataProperty mais également de FunctionalProperty qui permet donc de préciser que chaque animal a un un nom
    unique
    """
    class nomOfficiel(DataProperty, FunctionalProperty):
        domain = [Animal]
        range = [str]
        
    """
    Maintenant on va créer une ou deux classes par equivalence
    """
    class carnivore(Thing):
        equivalent_to = [Animal & mange.some(Animal)]
        def presente_toi(self):
            print("je suis un carnivore !")
            
            
    class herbivore(Thing):
        equivalent_to = [Animal & mange.only(Plante)]
        
    class nonsense(Thing):
        equivalent_to = [Animal & Plante]


A présent on peut commencer a créer nos individus et ensuite nous pourrons les manipuler en leur ajoutant des relations.

In [0]:
Lion = Animal()
Arbre = Plante()
Tigre = carnivore()
Brebis = Animal()

In [0]:
Brebis.mange = [Arbre]
Lion.mange = [Brebis]
Lion.nomOfficiel = "Lion"
Tigre.nomOfficiel = "Tigre"
Brebis.nomOfficiel = "Brebis"

In [0]:
onto.save(file = "out.owl", format = "ntriples")
!cat out.owl

## 4-Inférence et raisonnement 
A présent voyons comment nous pouvons ajouter des régles d'inférences afin d'inférer des nouvelles connaissances a partir des connaissances existantes 

In [0]:
with onto:
    rule = Imp()
    rule.set_as_rule("""carnivore(?a1), carnivore(?a2) -> nemangepas(?a1, ?a2)""")

In [0]:
"""
On execute les méthodes ici avant le raisonneur
"""
Lion.presente_toi()
Tigre.presente_toi()
Brebis.presente_toi()

In [0]:
with onto:
    sync_reasoner(infer_property_values = True)

In [0]:
"""
Voyons voire si les changement de classes affectent l'execution des méthodes
"""
Lion.presente_toi()
Tigre.presente_toi()
Brebis.presente_toi()

In [0]:
with onto:
    sync_reasoner_pellet(infer_property_values = True)

In [0]:
onto.save(file = "out.owl", format = "ntriples")
!cat out.owl
default_world.save()

In [0]:
import rdflib
graph = default_world.as_rdflib_graph()
requete = """
prefix : <https://urlinvente/monontologie#> 

SELECT DISTINCT ?nom
WHERE{
?s rdf:type :Animal . 
?s :nomOfficiel ?nom
}
"""
res = graph.query(requete)
for i in res:
    print(str(i[0]))

In [0]:
if Nothing in nonsense.equivalent_to:
    print("nonsense is inconsistent!")
if Nothing in Animal.equivalent_to:
    print("Animal is inconsistent!")